In [0]:
#Libraries import
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import csv
import pandas as pd
import os
from os import listdir
import re

In [0]:
#Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Load the model to predict
sess = tf.InteractiveSession()
model = load_model('/content/drive/My Drive/Data_Science/Dissertation/Inception_batch32_SGD_0.005_30epochs.log.hdf5')

In [0]:
# File directory
filepath = ('/content/drive/My Drive/Test_images/Dissertation_test')
food_list = ['apple pie','baby back ribs','bread pudding','breakfast burrito','caesar salad','chicken curry','chicken wings','club sandwich','cup cakes','dumplings','eggs benedict','filet mignon','fish and chips','french fries','french toast','greek salad','grilled cheese sandwich','grilled salmon','hamburger','hot dog','ice cream','macaroni and cheese','macarons','omelette','pancakes','pizza','pork chop','prime rib','risotto','spaghetti bolognese','spaghetti carbonara','steak']

In [0]:
#Remove digits from the file names
def remove_digits(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list
# Extract the dishes names from the file names
def prepare_data(filepath):
  #classes_images = defaultdict(list)
  global plate_content
  for subdir_path, dirs, files in os.walk(filepath):
    for file in files:
      path = os.path.join(subdir_path, file)
      print(path)
      file = file.replace("_"," ")
      class_name = file.split('.')
      class_name = class_name[0].split('+')
      img_raw = tf.read_file(path)
      image = tf.image.decode_image(img_raw).eval()
      sliding = sess.run(tf.shape(image))
      file_name = file
      plate_content = remove_digits(class_name)
      number_classes = subdir_path[-1]
      selecting(sliding,image,file_name,plate_content,number_classes)
      

In [0]:
#Iteratively change parameters
def selecting(sliding,image,file_name,plate_content,number_classes):
  global df_global
  for n in range(20,60,10):
    for m in range(40,110,20):
      ksize_rows, ksize_cols = int((sliding[0] * n)/100), int((sliding[1] * n)/100)
      strides_rows, strides_cols =  int((ksize_rows * m)/100), int((ksize_cols * m)/100)
      classes['slidind_window'] = n
      classes['strides'] = m
      classes['file_name'] = file_name
      classes['plate_content'] = plate_content
      classes['number_class'] = number_classes
      ksizes = [1, ksize_rows, ksize_cols, 1] # The size of sliding window
      strides = [1, strides_rows, strides_cols, 1]# How far the centers of 2 consecutive patches are in the image
      rates = [1, 1, 1, 1] # sample pixel consecutively
      padding='VALID' # or 'SAME'
      image_expanded = tf.expand_dims(image, 0)
      image_patches = tf.extract_image_patches(image_expanded, ksizes, strides, rates, padding)
      shapes = sess.run(tf.shape(image_patches))
      nr, nc = shapes[1], shapes[2]
      classes['patches_number'] = nr * nc
      for i in range(nr):
        for j in range(nc):
          x = sess.run(tf.reshape(image_patches[0,i,j,], [ksize_rows, ksize_cols, 3]))
          x = cv2.resize(x, dsize=(300, 300), interpolation=cv2.INTER_CUBIC)
          predict_class(model, x, False)
      df_local  = pd.DataFrame([classes], columns=classes.keys())
      df_global = df_global.append(df_local, sort=False)
      #print(df_global)
      classes.clear()
      classes["true_classes"] = 0

In [0]:
#Preprocess image, calling the predic method, find the winner class and match with lable
def predict_class(model, image, show_debug = False):
    img = np.expand_dims(image, axis=0)
    img = img / 255  
    pred = model.predict(img, verbose=0)
    #print(pred)
    max_prob = np.amax(pred)
    if max_prob > 0.6:
      #print(pred)
      index = np.argmax(pred)
      food_list.sort()
      pred_value = food_list[index]
      fill_dict(pred_value)
      fill_true_class(pred_value)
      if show_debug:
        plt.imshow(img[0])                           
        plt.axis('off')
        plt.title(pred_value)
        plt.show()

In [0]:
def fill_true_class(key):
  for i in plate_content:
    if key == i:
      classes["true_classes"] +=1

In [0]:
#Fill the dictionary with lables
def fill_dict(key):
  if key in classes:
    classes[key] += 1
  else:
    classes[key] = 1

In [0]:
#%%time
classes = {}
classes["true_classes"] = 0
true_patch = 0
df_global = pd.DataFrame()
prepare_data(filepath)
#print(true_classes)

In [0]:
FILE1 = '/content/drive/My Drive/DataFrames/df_1class.csv'
FILE2 = '/content/drive/My Drive/DataFrames/df_2class.csv'
FILE3 = '/content/drive/My Drive/DataFrames/df_3class.csv'
FILE4 = '/content/drive/My Drive/DataFrames/df_4class.csv'
df_1 = pd.read_csv(FILE1)
df_2 = pd.read_csv(FILE2)
df_3 = pd.read_csv(FILE3)
df_4 = pd.read_csv(FILE4)
OUTPUT_FILE = 'df_target.csv'

In [0]:
df_merged = pd.concat([df_1, df_2, df_3, df_4], ignore_index=True, sort=False)
df_merged = pd.DataFrame(df_merged)

In [5]:
df_merged.shape

(624, 39)

In [0]:
df_merged['Overall_classes_>0.6'] = df_merged.iloc[:,7:].sum(axis=1, skipna = True)

In [0]:
df_merged['all_classes'] = np.zeros(len(df_merged),dtype=float)
df_merged.fillna(0,inplace=True)

In [8]:
df_merged.head()

,true_classes,slidind_window,strides,file_name,plate_content,number_class,patches_number,macarons,apple pie,chicken wings,bread pudding,french toast,fish and chips,breakfast burrito,macaroni and cheese,cup cakes,hamburger,baby back ribs,steak,ice cream,pizza,french fries,hot dog,dumplings,grilled salmon,pork chop,chicken curry,spaghetti bolognese,risotto,eggs benedict,spaghetti carbonara,grilled cheese sandwich,omelette,caesar salad,prime rib,filet mignon,club sandwich,pancakes,greek salad,Overall_classes_>0.6,all_classes
0,21,20,40,apple pie.jpeg,['apple pie'],1,121,7.0,21.0,13.0,8.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,0.0
1,9,20,60,apple pie.jpeg,['apple pie'],1,49,4.0,9.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0
2,5,20,80,apple pie.jpeg,['apple pie'],1,36,2.0,5.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
3,2,20,100,apple pie.jpeg,['apple pie'],1,25,2.0,2.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
4,14,30,40,apple pie.jpeg,['apple pie'],1,36,2.0,14.0,4.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0


In [0]:
for count,i in enumerate(df_merged['plate_content']):
  if df_merged[eval(i)].iloc[count].all() > 0:
    df_merged.at[count,'all_classes'] = 1
  else:
    df_merged.at[count,'all_classes'] = 0

In [0]:
df_merged.to_csv('df_clean.csv')

In [0]:
indexNames = df_merged[df_merged['all_classes'] == 0 ].index
df_merged.drop(indexNames , inplace=True)

In [0]:
df_merged['Target_ratio'] = df_merged['true_classes']/df_merged['Overall_classes_>0.6']

In [13]:
df_merged.shape

(256, 42)

In [14]:
df_pivot = df_merged['Target_ratio'].groupby([df_merged['number_class'],df_merged['file_name']]).max()
df_pivot = pd.DataFrame(df_pivot)
df_pivot.head(39)

Target_ratio
number_class file_name                                                      
1            apple pie.jpeg                                         1.000000
             bread pudding.jpg                                      1.000000
             breakfast burrito.jpg                                  1.000000
             chicken wings.jpg                                      1.000000
             cup cakes.jpeg                                         1.000000
             dumplings.jpg                                          1.000000
             omelette.jpg                                           1.000000
             risotto.jpg                                            0.833333
             spaghetti bolognese.jpg                                0.666667
             spaghetti carbonara.jpg                                1.000000
2            apple pie+ice cream.jpg                                1.000000
             apple pie+ice cream2.jpg                               0.500000
             apple pie+ice cream3.jpeg                              1.000000
             club sandwich+caesar salad.jpg                         1.000000
             hamburger+caesar salad.jpg                             1.000000
             risotto+chicken wings.jpg                              1.000000
             risotto+eggs benedict.jpg                              1.000000
             steak+risotto.jpg                                      1.000000
3            baby back ribs+caesar salad+french fries.jpg           0.708333
             club sandwich+french fries+greek salad.jpg             0.833333
             filet mignon+french fries+greek salad.jpg              0.714286
             french fries+hamburger+eggs benedict.jpg               1.000000
             french fries+hot dog+chicken wings.jpg                 0.857143
             french fries+steak+caesar salad.jpg                    0.900000
             steak+greek salad+french fries.jpg                     1.000000
4            steak+eggs benedict+french fries+greek salad2.jpg      0.520000
             steak+eggs benedict+french fries+greek salad3.jpg      0.615385
             steak+eggs benedict+french fries+greek salad4.jpg      0.444444

In [0]:
df_final.to_csv('df_final.csv')
df_merged.to_csv('df_merged.csv')
df_pivot.to_csv('df_pivot.csv')

In [0]:
df_final = pd.merge(df_pivot, df_merged, how = 'inner', on=['file_name','Target_ratio'])

In [0]:
df_final.to_csv('df_final.csv')

In [0]:
df_temt = df_final[['slidind_window','strides','Target_ratio']]
df_temt.head(200)

In [0]:
df_final['sliding/stride'] = df_final["slidind_window"].map(str) + "/" + df_final["strides"].map(str)
df_final.head()

In [19]:
df_final['sliding/stride'].groupby([df_final['number_class'],df_final['sliding/stride']]).count()

number_class  sliding/stride
1             20/100            1
              30/100            3
              30/40             1
              30/60             1
              30/80             2
              40/100            7
              40/40             6
              40/60             6
              40/80             7
              50/100            8
              50/40             8
              50/60             8
              50/80             8
2             30/100            2
              30/60             2
              30/80             2
              40/100            4
              40/40             4
              40/60             4
              40/80             5
              50/100            5
              50/40             5
              50/60             4
              50/80             5
3             20/100            1
              20/60             2
              30/60             1
              30/80             2
              40/40